In [1]:
from itertools import chain, tee
from typing import (
    Any,
    Dict,
    List,
    Optional,
    Sequence,
    Tuple,
    Iterator,
    TypeVar,
    Callable,
)
from dataclasses import dataclass, field
import sys
sys.path.append("../")
from iterparse import *

In [2]:
@dataclass
class XMLElement:
    name: str
    attributes: Dict[str,str]
    children: List["XMLElement"] = field(default_factory=lambda: [])
    text: Optional[str] = None


In [8]:
underscore = term("_")

element_name = join(
    try_choice(
        alpha,
        term("_")),
    zero_or_more(
        try_choice(
            alpha,
            numeric,
            term_set("_.-")))
)
element_name = transform(element_name, lambda x: x[0]+"".join(x[1]))

attribute_name = element_name

attribute_value = join(
    term('"'),
    zero_or_more(negated(term_backtrack('"'))),
    term('"')
)
attribute_value = transform(attribute_value, lambda x: "".join(x[1]))

attribute_key_value = join(
    attribute_name,
    term("="),
    attribute_value
)
attribute_key_value = transform(attribute_key_value, lambda x: (x[0],x[2]))

self_closing_element = join(
    term("<"),
    element_name,
    zero_or_more(
        skip_left(
            one_or_more(space),
            attribute_key_value
        )
    ),
    term("/"),
    term(">")
)
self_closing_element = transform(
    self_closing_element, 
    lambda x: XMLElement(name=x[1],attributes={k:v for k,v in x[2]})
)

opening_tag = join(
    term("<"),
    element_name,
    zero_or_more(
        join(
            one_or_more(space),
            attribute_key_value
        )
    ),
    term(">")
)

# TODO: this should not work
self_closing_element(iter('<afdsfq a/>'))

([], XMLElement(name='afdsfq', attributes={}, children=[], text=None))